In [1]:
!pip install numpy
!pip install matplotlib
!pip install pandas
!pip install seaborn
!pip install plotly


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [3]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

## Читання та перегляд набору даних

In [4]:
ds = pd.read_csv("../data/dataset.csv")
ds.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0,satisfied


## Data engineering 

Missing data imputation

In [5]:
# створимо функцію для заповнення відсутніх значень:
def impute_na(df, variable, value):
    return df[variable].fillna(value)

In [6]:
# Знаходження медіани для стовпця
median_arrival_delay = ds['Arrival Delay in Minutes'].median()

In [7]:
# Заміна пропущених значень на медіану
ds['Arrival Delay in Minutes'] = impute_na(ds, 'Arrival Delay in Minutes', median_arrival_delay)

In [8]:
for col in ds.columns:
    if ds[col].isnull().values.any():
        print("Missing data in ", col, ds[col].isnull().sum())
    else:
        print("There aren't missing data")

There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data
There aren't missing data


## Categorical encoding

In [9]:
# for one hot encoding with sklearn
from sklearn.preprocessing import OneHotEncoder

In [10]:
ds.nunique()

Unnamed: 0                           103904
id                                   129880
Gender                                    2
Customer Type                             2
Age                                      75
Type of Travel                            2
Class                                     3
Flight Distance                        3821
Inflight wifi service                     6
Departure/Arrival time convenient         6
Ease of Online booking                    6
Gate location                             6
Food and drink                            6
Online boarding                           6
Seat comfort                              6
Inflight entertainment                    6
On-board service                          6
Leg room service                          6
Baggage handling                          5
Checkin service                           6
Inflight service                          6
Cleanliness                               6
Departure Delay in Minutes      

In [11]:
#categorics
ds[["Gender","Customer Type","Type of Travel","Class","satisfaction"]].head()

,Gender,Customer Type,Type of Travel,Class,satisfaction
0,Male,Loyal Customer,Personal Travel,Eco Plus,neutral or dissatisfied
1,Male,disloyal Customer,Business travel,Business,neutral or dissatisfied
2,Female,Loyal Customer,Business travel,Business,satisfied
3,Female,Loyal Customer,Business travel,Business,neutral or dissatisfied
4,Male,Loyal Customer,Business travel,Business,satisfied


Label encoding

In [12]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

ds["Gender"] = le.fit_transform(ds["Gender"])
ds["Customer Type"] = le.fit_transform(ds["Customer Type"])
ds["Type of Travel"] = le.fit_transform(ds["Type of Travel"])
ds["Class"] = le.fit_transform(ds["Class"])
ds["satisfaction"] = le.fit_transform(ds["satisfaction"])

## Outlier Engineering

In [13]:
from sklearn.neighbors import LocalOutlierFactor

numerical_columns = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']

# Ініціалізація моделі LocalOutlierFactor
lof = LocalOutlierFactor(contamination=0.05)  # Встановіть contamination залежно від розміру викидів, які ви очікуєте

# Обчислення оцінки аномалій (викидів) для кожного прикладу в датасеті
outlier_scores = lof.fit_predict(ds[numerical_columns])

# Відображення кількості викидів
outlier_count = len(ds[outlier_scores == -1])
print("Number of outliers:", outlier_count)

# Видалення рядків, що містять викиди
ds = ds[outlier_scores != -1]

# Відображення оновленого датасету без викидів
print(f"Dataset without outliers:{len(ds)}")

Number of outliers: 6494
Dataset without outliers:123386


In [14]:
ds.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,1,0,13,1,2,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0,0
1,1,5047,1,1,25,0,0,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0,0
2,2,110028,0,0,26,0,0,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0,1
3,3,24026,0,0,25,0,0,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0,0
4,4,119299,1,0,61,0,0,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0,1


## Model tuning


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

Define target and features columns

In [16]:
y_column = 'satisfaction' # target variable
X_columns = [
    "Gender", "Customer Type", "Age", "Type of Travel", "Class",
    "Flight Distance", "Inflight wifi service", "Departure/Arrival time convenient",
    "Ease of Online booking", "Gate location", "Food and drink",
    "Online boarding", "Seat comfort", "Inflight entertainment",
    "On-board service", "Leg room service", "Baggage handling",
    "Checkin service", "Inflight service", "Cleanliness",
    "Departure Delay in Minutes", "Arrival Delay in Minutes"
]
X = ds[X_columns]
y = ds[y_column]

Train test set split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(98708, 22)
(98708,)
(24678, 22)
(24678,)


Побудова базової моделі випадкового лісу

In [18]:
rf = RandomForestClassifier(max_features=5, n_estimators=100)

In [19]:
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [20]:
print('test set metrics: ', metrics.classification_report(y_test, y_pred))

test set metrics:                precision    recall  f1-score   support

           0       0.96      0.98      0.97     14039
           1       0.98      0.94      0.96     10639

    accuracy                           0.96     24678
   macro avg       0.97      0.96      0.96     24678
weighted avg       0.97      0.96      0.96     24678



In [21]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 5,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

## Налаштування гіперпараметрів

n_estimators = кількість дерев у лісі

max_features = максимальна кількість функцій, які розглядаються для розділення вузла

max_depth = максимальна кількість рівнів у кожному дереві рішень

min_samples_split = мінімальна кількість точок даних, розміщених у вузлі до того, як вузол буде розділено

min_samples_leaf = мінімальна кількість точок даних, дозволених у листовому вузлі

bootstrap = метод вибірки точок даних (із або без заміни)

У методі пошуку за сіткою ми створюємо сітку можливих значень для гіперпараметрів. Кожна ітерація спробує комбінацію гіперпараметрів в певному порядку. Модель обчислюється на кожній можливій комбінації гіперпараметрів і реєструє продуктивність моделі. Нарешті, повертає найкращу модель з найкращими гіперпараметрами.

In [22]:
%%time
# Create the random grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_features': [0.5, 0.6, 0.8],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 4],
    'bootstrap': [False]
}



rf = RandomForestClassifier()

grid = GridSearchCV(estimator=rf, 
                    param_grid=param_grid, 
                    scoring='precision', 
                    cv=5,
                    verbose = 3,
                    return_train_score=True)

grid.fit(X_train, y_train)

print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5] END bootstrap=False, max_features=0.5, min_samples_leaf=2, min_samples_split=5, n_estimators=50;, score=(train=1.000, test=0.970) total time=  21.7s
[CV 2/5] END bootstrap=False, max_features=0.5, min_samples_leaf=2, min_samples_split=5, n_estimators=50;, score=(train=1.000, test=0.971) total time=  22.0s
[CV 3/5] END bootstrap=False, max_features=0.5, min_samples_leaf=2, min_samples_split=5, n_estimators=50;, score=(train=1.000, test=0.973) total time=  22.8s
[CV 4/5] END bootstrap=False, max_features=0.5, min_samples_leaf=2, min_samples_split=5, n_estimators=50;, score=(train=1.000, test=0.971) total time=  22.3s
[CV 5/5] END bootstrap=False, max_features=0.5, min_samples_leaf=2, min_samples_split=5, n_estimators=50;, score=(train=1.000, test=0.972) total time=  22.7s
[CV 1/5] END bootstrap=False, max_features=0.5, min_samples_leaf=2, min_samples_split=5, n_estimators=100;, score=(train=1.000, test=0.971) total ti

[CV 3/5] END bootstrap=False, max_features=0.5, min_samples_leaf=4, min_samples_split=10, n_estimators=100;, score=(train=0.998, test=0.974) total time=  42.5s
[CV 4/5] END bootstrap=False, max_features=0.5, min_samples_leaf=4, min_samples_split=10, n_estimators=100;, score=(train=0.998, test=0.972) total time=  40.3s
[CV 5/5] END bootstrap=False, max_features=0.5, min_samples_leaf=4, min_samples_split=10, n_estimators=100;, score=(train=0.997, test=0.975) total time=  43.4s
[CV 1/5] END bootstrap=False, max_features=0.5, min_samples_leaf=4, min_samples_split=10, n_estimators=150;, score=(train=0.997, test=0.973) total time= 1.1min
[CV 2/5] END bootstrap=False, max_features=0.5, min_samples_leaf=4, min_samples_split=10, n_estimators=150;, score=(train=0.998, test=0.974) total time=  58.4s
[CV 3/5] END bootstrap=False, max_features=0.5, min_samples_leaf=4, min_samples_split=10, n_estimators=150;, score=(train=0.997, test=0.974) total time= 1.0min
[CV 4/5] END bootstrap=False, max_featur

[CV 5/5] END bootstrap=False, max_features=0.6, min_samples_leaf=4, min_samples_split=5, n_estimators=150;, score=(train=0.998, test=0.973) total time= 1.2min
[CV 1/5] END bootstrap=False, max_features=0.6, min_samples_leaf=4, min_samples_split=10, n_estimators=50;, score=(train=0.997, test=0.973) total time=  25.6s
[CV 2/5] END bootstrap=False, max_features=0.6, min_samples_leaf=4, min_samples_split=10, n_estimators=50;, score=(train=0.998, test=0.972) total time=  23.8s
[CV 3/5] END bootstrap=False, max_features=0.6, min_samples_leaf=4, min_samples_split=10, n_estimators=50;, score=(train=0.998, test=0.973) total time=  25.1s
[CV 4/5] END bootstrap=False, max_features=0.6, min_samples_leaf=4, min_samples_split=10, n_estimators=50;, score=(train=0.998, test=0.971) total time=  23.2s
[CV 5/5] END bootstrap=False, max_features=0.6, min_samples_leaf=4, min_samples_split=10, n_estimators=50;, score=(train=0.998, test=0.973) total time=  24.4s
[CV 1/5] END bootstrap=False, max_features=0.6

[CV 2/5] END bootstrap=False, max_features=0.8, min_samples_leaf=4, min_samples_split=5, n_estimators=100;, score=(train=0.998, test=0.971) total time= 1.1min
[CV 3/5] END bootstrap=False, max_features=0.8, min_samples_leaf=4, min_samples_split=5, n_estimators=100;, score=(train=0.998, test=0.972) total time= 1.0min
[CV 4/5] END bootstrap=False, max_features=0.8, min_samples_leaf=4, min_samples_split=5, n_estimators=100;, score=(train=0.998, test=0.969) total time= 1.0min
[CV 5/5] END bootstrap=False, max_features=0.8, min_samples_leaf=4, min_samples_split=5, n_estimators=100;, score=(train=0.998, test=0.971) total time= 1.0min
[CV 1/5] END bootstrap=False, max_features=0.8, min_samples_leaf=4, min_samples_split=5, n_estimators=150;, score=(train=0.998, test=0.970) total time= 1.5min
[CV 2/5] END bootstrap=False, max_features=0.8, min_samples_leaf=4, min_samples_split=5, n_estimators=150;, score=(train=0.998, test=0.971) total time= 1.5min
[CV 3/5] END bootstrap=False, max_features=0.8

Найкращі параметри: {'bootstrap': False, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 150} з оцінкою 0.97.

In [23]:
grid_results = pd.concat([pd.DataFrame(grid.cv_results_["params"]),
                          pd.DataFrame(grid.cv_results_["mean_test_score"], 
                          columns=["precision"])],
                          axis=1)

grid_results

,bootstrap,max_features,min_samples_leaf,min_samples_split,n_estimators,precision
0,False,0.5,2,5,50,0.971504
1,False,0.5,2,5,100,0.971878
2,False,0.5,2,5,150,0.972074
3,False,0.5,2,10,50,0.973029
4,False,0.5,2,10,100,0.973119
5,False,0.5,2,10,150,0.973695
6,False,0.5,4,5,50,0.972743
7,False,0.5,4,5,100,0.973241
8,False,0.5,4,5,150,0.973398
9,False,0.5,4,10,50,0.972638


In [24]:
grid_contour = grid_results.groupby([  'max_features', 'min_samples_leaf',
       'min_samples_split', 'n_estimators']).mean()
grid_contour

bootstrap  \
max_features min_samples_leaf min_samples_split n_estimators              
0.5          2                5                 50                  0.0   
                                                100                 0.0   
                                                150                 0.0   
                              10                50                  0.0   
                                                100                 0.0   
                                                150                 0.0   
             4                5                 50                  0.0   
                                                100                 0.0   
                                                150                 0.0   
                              10                50                  0.0   
                                                100                 0.0   
                                                150                 0.0   
0.6          2                5                 50                  0.0   
                                                100                 0.0   
                                                150                 0.0   
                              10                50                  0.0   
                                                100                 0.0   
                                                150                 0.0   
             4                5                 50                  0.0   
                                                100                 0.0   
                                                150                 0.0   
                              10                50                  0.0   
                                                100                 0.0   
                                                150                 0.0   
0.8          2                5                 50                  0.0   
                                                100                 0.0   
                                                150                 0.0   
                              10                50                  0.0   
                                                100                 0.0   
                                                150                 0.0   
             4                5                 50                  0.0   
                                                100                 0.0   
                                                150                 0.0   
                              10                50                  0.0   
                                                100                 0.0   
                                                150                 0.0   

                                                              precision  
max_features min_samples_leaf min_samples_split n_estimators             
0.5          2                5                 50             0.971504  
                                                100            0.971878  
                                                150            0.972074  
                              10                50             0.973029  
                                                100            0.973119  
                                                150            0.973695  
             4                5                 50             0.972743  
                                                100            0.973241  
                                                150            0.973398  
                              10                50             0.972638  
                                                100            0.973492  
                                                150            0.973576  
0.6          2                5                 50             0.971041  
                                                100            0.971478  
                           